For the theoretical background regarding these techniques , pleases refer to theoretical discussion on linear models. 

Now we'll discuss a case study solution using multiple linear regression, and regularised linear regresison [Ridge and Lasso] . We'll also look at hyper parameter tuning for regularised regression.

A little background on the case study. This data belongs to a loan aggregator agency which connects loan applications to different financial institutions in attempt to get the best interest rate. They want to now utilise past data to predict interest rate given by any financial institute just by looking at loan application characteristics.

To achieve that , they have decided to do a POC with a data from a particular financial institution. The data is given in the file "loans data.csv". Lets begin: 

In [1]:


import pandas as pd
import math
from sklearn.model_selection_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import numpy as np
from sklearn.model_selection import KFold
%matplotlib inline


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
data_file=r'D:/Data/loans data.csv'
ld=pd.read_csv(data_file)

In [ ]:
ld.head()

In [ ]:
ld.shape

You can see that variable Interest.Rate and Debt.To.Income.Ratio contain "%" sign in their values and because of which they have come as character columns in the data. Lets remove these percentages first.

In [ ]:
for col in ["Interest.Rate","Debt.To.Income.Ratio"]:
    ld[col]=ld[col].astype("str")
    ld[col]=[x.replace("%","") for x in ld[col]]



In [ ]:
ld.head()

In [ ]:
ld.dtypes

We can see that many columns which should have really been numbers have been imported as character columns , probably because some characters values in those columns in the files. We'll convert all such columns to numbers .

In [ ]:
for col in ["Amount.Requested","Amount.Funded.By.Investors","Open.CREDIT.Lines","Revolving.CREDIT.Balance",
           "Inquiries.in.the.Last.6.Months","Interest.Rate","Debt.To.Income.Ratio"]:
    ld[col]=pd.to_numeric(ld[col],errors="coerce")

In [ ]:
ld.dtypes

Next we will make dummy variables for remaining categorical variables

In [ ]:
ld["Loan.Length"].value_counts()

Function get_dummies creates dummy variables for all the categories present in the categorical variable. Result is a dataframe, we can then choose and drop the dummies that we want to drop and attach the ones selected back to our original data.

In [ ]:
ll_dummies=pd.get_dummies(ld["Loan.Length"])

In [ ]:
ll_dummies.head()

We'll add dummy variable for "36 months" to our data and ignore the rest two. 

In [ ]:
ld["LL_36"]=ll_dummies["36 months"]

Now that we'are done with dataframe ll_dummies , we can drop it. Below we demonstrate a general way of removing variables from notebook environment.

In [ ]:
ld.head()

In [ ]:
%reset_selective ll_dummies

To know what all variables are in the environment, you can use function "who"

In [ ]:
who

Now that we have created dummies for Loan.Length, we need to remove this from the dataframe.

In [ ]:
ld=ld.drop('Loan.Length',axis=1)

In [ ]:
ld.dtypes

Next we examine variable "Loan.Purpose".

In [ ]:
ld["Loan.Purpose"].value_counts()

There are 14 categories in that variable, we can either make 13 dummies or we can club few of these categories together and reduce the number of effective categories and then make dummy variables for those.

It makes sense to club those categories which behave similarly in terms of their effect on response. Or in other words , we can club those categories for which average interest rates are similar in the data.

In [ ]:
round(ld.groupby("Loan.Purpose")["Interest.Rate"].mean())

We can see from the table above that there are 4 effective categoris in the data. Lets club them

In [ ]:
for i in range(len(ld.index)):
    if ld["Loan.Purpose"][i] in ["car","educational","major_purchase"]:
        ld.loc[i,"Loan.Purpose"]="cem"
    if ld["Loan.Purpose"][i] in ["home_improvement","medical","vacation","wedding"]:
        ld.loc[i,"Loan.Purpose"]="hmvw"
    if ld["Loan.Purpose"][i] in ["credit_card","house","other","small_business"]:
        ld.loc[i,"Loan.Purpose"]="chos"
    if ld["Loan.Purpose"][i] in ["debt_consolidation","moving"]:
        ld.loc[i,"Loan.Purpose"]="dm"

Now we make dummies for this variable

In [ ]:
lp_dummies=pd.get_dummies(ld["Loan.Purpose"],prefix="LP")

In [ ]:
lp_dummies.head()

We'll add this data to original data. And then drop original variable "Loan.Purpose" and one of the dummies "LP_renewable_energy"

In [ ]:
ld=pd.concat([ld,lp_dummies],1)
ld=ld.drop(["Loan.Purpose","LP_renewable_energy"],1)

In [ ]:
ld.dtypes

Next we look at variable "State".

In [ ]:
ld["State"].nunique()

There are too many unique values. Although its not a legit reason to drop a variable. But we'll ignore this in this discussion any way in order to reduce amount of data prep that we are doing here. You can try including it in the model and see if the performance improves.

In [ ]:
ld=ld.drop(["State"],1)

Next we take care of variable Home.Ownership.

In [ ]:
ld["Home.Ownership"].value_counts()

In [ ]:
ld["ho_mort"]=np.where(ld["Home.Ownership"]=="MORTGAGE",1,0)
ld["ho_rent"]=np.where(ld["Home.Ownership"]=="RENT",1,0)
ld=ld.drop(["Home.Ownership"],1)

We have simply ignored values OTHER and NONE , and considered that there are only 3 categories and created only two dummies . We did this because of very low frequencies of OTHER and NONE

In [ ]:
ld["FICO.Range"].head()

If you look at first few values of variable FICO.Range , you can see that we can convert it to numeric by taking average of the range given. To do that first we need to split the column with "-", so that we can have both end of ranges in separate columns and then we can simply average them.
Lets first split.

In [ ]:
ld['f1'] = ld['FICO.Range'].str.split("-",expand=True)[0]
ld['f2'] = ld['FICO.Range'].str.split("-",expand=True)[1]

In [ ]:
# ld['f1'], ld['f2'] = zip(*ld['FICO.Range'].apply(lambda x: x.split('-', 1)))

Now we create new variable "fico" by averaging f1 and f2. And then we'll drop the original variable FICO.Range and f1,f2.

In [ ]:
ld["fico"]=0.5*(pd.to_numeric(ld["f1"])+pd.to_numeric(ld["f2"]))

ld=ld.drop(["FICO.Range","f1","f2"],1)

Next we look at variavle Employment.Length. You'll see that we can convert that to number as well.

In [ ]:
ld.dtypes

In [ ]:
ld["Employment.Length"].value_counts()

In [ ]:
ld["Employment.Length"]=ld["Employment.Length"].astype("str")
ld["Employment.Length"]=[x.replace("years","") for x in ld["Employment.Length"]]
ld["Employment.Length"]=[x.replace("year","") for x in ld["Employment.Length"]]

In [ ]:
ld["Employment.Length"].value_counts()

We can convert everything else to numbers , but "Unk" are a problem. We can look at average interest across all values of Employment.Length and then replace "Unk" with value which has closet average response.

In [ ]:
round(ld.groupby("Employment.Length")["Interest.Rate"].mean(),2)

As you can see "Unk" is similar to "< 1".

In [ ]:
ld["Employment.Length"]=[x.replace("Unk","< 1") for x in ld["Employment.Length"]]
ld["Employment.Length"]=[x.replace("10+","10") for x in ld["Employment.Length"]]
ld["Employment.Length"]=[x.replace("< 1","0") for x in ld["Employment.Length"]]
ld["Employment.Length"]=pd.to_numeric(ld["Employment.Length"],errors="coerce")

In [ ]:
ld.dtypes

Now we have all the variables as numbers. After dropping observations with missing values , we can proceed to build oru model.

In [ ]:
ld.shape

In [ ]:
ld.dropna(axis=0,inplace=True)

In [ ]:
ld.shape

We now split our data into two random parts . One to build model on , Another to test its performance. Option "random_state" is used to make our random operation reproducible.

In [ ]:
ld_train, ld_test = train_test_split(ld, test_size = 0.2,random_state=2)

In [ ]:
print('Train -', ld_train.shape)
print('Test  -', ld_test.shape)

In [ ]:
lm=LinearRegression()

Above line creates and object of class LinearRegression named lm. We can use this object to access all functions realted to LinearRegression.

Now we'll separate predictors and response for both the datasets . We'll also drop ID from predictor's list because it doesnt make sense to include an ID variable in the model. Variable "Amount.Funded.By.Investors" will also be dropped because it wont be available until the loan has been processed. We can use only those variables which are present at the point of the business process where we want to apply our model.

In [ ]:
x_train=ld_train.drop(["Interest.Rate","ID","Amount.Funded.By.Investors"],1)
y_train=ld_train["Interest.Rate"]
x_test=ld_test.drop(["Interest.Rate","ID","Amount.Funded.By.Investors"],1)
y_test=ld_test["Interest.Rate"]

In [ ]:
x_train.head()

In [ ]:
y_train.head()

Now we can fit our model using lm the LinearRegression object that we created earlier

In [ ]:
lm.fit(x_train,y_train)

Next we predict resposne on our test data , calculate errors on those prediction and then rmse for those residuals. That is the measure of performance on the test data. We can use this measure to compare other models that we'll build.

In [ ]:
p_test=lm.predict(x_test)
view_act_pred = pd.concat([pd.Series(y_test).reset_index(drop=True), pd.Series(p_test)], axis=1)
view_act_pred.columns = ['Actual', 'Pred']
view_act_pred['Error'] = view_act_pred.Actual - view_act_pred.Pred
view_act_pred.head()

In [ ]:
residual=y_test-p_test

rmse_lm=np.sqrt(np.dot(residual,residual)/len(p_test))

rmse_lm

We can use this to compare our linear regression model with other techniques and evenutall pick the one with least error .

Next we show how to extract coefficient produced by our model

In [ ]:
lm.coef_

In [ ]:
coefs=lm.coef_

features=x_train.columns

list(zip(features,coefs))

In [ ]:
lm.intercept_

We can see that linear regression has produced coefficients for all variables. If you recall our theoretical discussion, we need to penalise coefficient for the variables which are not really contributing well to our resposne and might be causing overfitting of the model. Among the regularised technique we'll first look at Ridge regression.

Since penalty in ridge regression is a hyperparameter , we'd look at multiple values of it and choose the best one through 10 fold cross validation.

In [ ]:
# Finding best value of penalty weight with cross validation for ridge regression
alphas=np.linspace(.0001,10,100)
# We need to reset index for cross validation to work without hitch
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [ ]:
x_train.shape

In [ ]:
rmse_list=[]
for a in alphas:
    ridge = Ridge(fit_intercept=True, alpha=a)

    # computing average RMSE across 10-fold cross validation
#     kf = KFold(len(x_train), n_folds=10, random_state=2)
    kf = KFold(n_splits=10, random_state=2)
    xval_err = 0
#     for train, test in kf:
    for train, test in kf.split(x_train):
        ridge.fit(x_train.loc[train], y_train[train])
        p = ridge.predict(x_train.loc[test])
        err = y_train[test] - p
        xval_err += np.dot(err,err)
    rmse_10cv = np.sqrt(xval_err/len(x_train))
    # uncomment below to print rmse values for individidual alphas
#     print('{:.3f}\t {:.6f}\t '.format(a,rmse_10cv))
    rmse_list.extend([rmse_10cv])
best_alpha=alphas[rmse_list==min(rmse_list)]
print('Alpha with min 10cv error is : ',best_alpha )

best value of alpha  might be slightly different across different runs because of random nature of cross validation. So dont worry if you determine a different value of best alpha.

Next we fit Ridge Regression on the entire train data with best value of alpha we just determined.


In [ ]:

ridge=Ridge(fit_intercept=True,alpha=best_alpha)

ridge.fit(x_train,y_train)

In [ ]:
list(zip(x_train.columns,ridge.coef_))

In [ ]:
p_test=ridge.predict(x_test)

residual=y_test - p_test

rmse_ridge=np.sqrt(np.dot(residual,residual)/len(p_test))

rmse_ridge

You can see that ridge regression though, shrinks the coefficients but never makes them exactly zero, essentially never reduce our model size. Next we look at lasso Regression.

In [ ]:
alphas=np.linspace(0.0001,1,100)
rmse_list=[]
for a in alphas:
    lasso = Lasso(fit_intercept=True, alpha=a,max_iter=10000)

    # computing RMSE using 10-fold cross validation
#     kf = KFold(len(x_train), n_folds=10)
    kf = KFold(n_splits=10, random_state=2)
    xval_err = 0
#     for train, test in kf:
    for train, test in kf.split(x_train):
        lasso.fit(x_train.loc[train], y_train[train])
        p =lasso.predict(x_train.loc[test])
        err = p - y_train[test]
        xval_err += np.dot(err,err)
    rmse_10cv = np.sqrt(xval_err/len(x_train))
    rmse_list.extend([rmse_10cv])
    # Uncomment below to print rmse values of individual alphas
#     print('{:.3f}\t {:.4f}\t '.format(a,rmse_10cv))
best_alpha=alphas[rmse_list==min(rmse_list)]
print('Alpha with min 10cv error is : ',best_alpha )

In [ ]:
lasso=Lasso(fit_intercept=True,alpha=best_alpha)

lasso.fit(x_train,y_train)

In [ ]:
list(zip(x_train.columns,lasso.coef_))

In [ ]:
p_test=lasso.predict(x_test)

residual=p_test-y_test

rmse_lasso=np.sqrt(np.dot(residual,residual)/len(p_test))

rmse_lasso

In [ ]:
# Testing on a new record / Scoring

scoring = pd.read_csv('D:/Data/loans_data_scoring.csv')
scoring

In [ ]:
scoring['Predicted'] = lasso.predict(scoring)
scoring

We can see that lasso regression, not only improves performance on the data slightly , but also makes size of the model smaller by making many coefficents exactly zero, thus excluding them from our model.

In [ ]:
import math

In [ ]:
P=0.001
P/(1-P)
math.log(P/(1-P))

In [ ]:
b0= 0.8
b1 = 0.5
x1 = 1
z=b0+b1*x1
print(z)
print(math.exp(z)/(1+math.exp(z)))

### Logistic Model for Binary Classification

A retail banking institution is going to float a stock trading facility for their existing customer. Since this kind of facitlity is nothing new , company knows that they will have to incetivise their customers for adopting their offerings. One way to incetiwise is to offer discounts on the commision for trading transactions.

One issue with that is that only about 10% of the customers do enought trades for earnings after discounts to be profitable. Company wants to figure out, which are those 10% customer so that it can selectively offer them discount. there is no magic way to figure that out. So company rolled out this service to about 10000+ of their customers and observed their trading behaviour for 6 months and after that they labelled them into two revenue.grids 1 and 2.
using this data, now they want us to build a classification model which can be used to classify their remaining customers into these revenue grids.



#### Logistic Regression from Scikit Learn 

Logistic Regression in scikit learn already contains penalties. l1 and l2 [Read as L-one & L-two] penalties . l1 penalty is same as lasso penalty where as l2 is same as ridge penalty. parameter C for logistic regression function is the hyperparameter for penalty . However it works in inverse fashion, i.e. if C takes smalle , it means higher penalty. 

For the case that we have discussed here , we have discussed l1 penalty with value of C as 1. We have left following things for you to try on your own.

* model with l2 penalty
* Finding optimal value of hyperparameter C with cross-validation for both the penalties

You will find these in the practice exercise as well

you can use auc value obtained from function roc_auc_score to select best value for the hyperparameter. Higher the auc, better is the model . If you dont recall this, please go back to the theoretical reading material.

Lets beging our model building process.


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [16]:
data_file=r'D:/Data/Existing Base.csv'
bd=pd.read_csv(data_file)

In [17]:
bd.head()

,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,...,Investment Tax Saving Bond,Home Loan,Online Purchase Amount,Revenue Grid,gender,region,Investment in Commudity,Investment in Equity,Investment in Derivative,Portfolio Balance
0,1,Zero,51-55,Partner,Manual Worker,Secretarial/Admin,Own Home,"<17,500, >=15,000",No,No,...,19.99,0.00,0.00,1,Female,Wales,74.67,18.66,32.32,89.43
1,2,Zero,55-60,Single/Never Married,Retired,Retired,Own Home,"<27,500, >=25,000",No,No,...,0.00,0.00,0.00,2,Female,North West,20.19,0.00,4.33,22.78
2,3,Zero,26-30,Single/Never Married,Professional,Other,Own Home,"<30,000, >=27,500",Yes,No,...,0.00,3.49,0.00,2,Male,North,98.06,31.07,80.96,171.78
3,5,Zero,18-21,Single/Never Married,Professional,Manual Worker,Own Home,"<15,000, >=12,500",No,No,...,0.00,0.00,0.00,2,Female,West Midlands,4.10,14.15,17.57,-41.70
4,6,Zero,45-50,Partner,Business Manager,Unknown,Own Home,"<30,000, >=27,500",No,No,...,0.00,45.91,25.98,2,Female,Scotland,70.16,55.86,80.44,235.02


In [18]:
bd["children"].value_counts()

Zero    6208
1       1848
2       1607
3        473
4+        19
Name: children, dtype: int64

It seems we can directly convert this to numeric.

In [19]:
bd.loc[bd["children"]=="Zero","children"]="0"
bd.loc[bd["children"]=="4+","children"]="4"
bd["children"]=pd.to_numeric(bd["children"],errors="coerce")

In [20]:
bd["Revenue Grid"].value_counts()

2    9069
1    1086
Name: Revenue Grid, dtype: int64

In [21]:
bd["y"]=np.where(bd["Revenue Grid"]==2,0,1)
bd=bd.drop(["Revenue Grid"],1)

For variable , age_band if we treat it as categorical variable , we can combine its categories by looking average response rate across its categories.

In [22]:
bd["age_band"].value_counts()

45-50      1359
36-40      1134
41-45      1112
31-35      1061
51-55      1052
55-60      1047
26-30       927
61-65       881
65-70       598
22-25       456
71+         410
18-21        63
Unknown      55
Name: age_band, dtype: int64

In [23]:
round(bd.groupby("age_band")["y"].mean(),2)

age_band
18-21      0.17
22-25      0.11
26-30      0.11
31-35      0.11
36-40      0.13
41-45      0.11
45-50      0.10
51-55      0.10
55-60      0.11
61-65      0.09
65-70      0.10
71+        0.10
Unknown    0.05
Name: y, dtype: float64

In [24]:
for i in range(len(bd)):
    if bd["age_band"][i] in ["71+","65-70","51-55","45-50"]:
        bd.loc[i,"age_band"]="ab_10"
    if bd["age_band"][i] in ["55-60","41-45","31-35","22-25","26-30"]:
        bd.loc[i,"age_band"]="ab_11"
    if bd["age_band"][i]=="36-40":
        bd.loc[i,"age_band"]="ab_13"
    if bd["age_band"][i]=="18-21":
        bd.loc[i,"age_band"]="ab_17"
    if bd["age_band"][i]=="61-65":
        bd.loc[i,"age_band"]="ab_9"
ab_dummies=pd.get_dummies(bd["age_band"])
ab_dummies.head()

,Unknown,ab_10,ab_11,ab_13,ab_17,ab_9
0,0,1,0,0,0,0
1,0,0,1,0,0,0
2,0,0,1,0,0,0
3,0,0,0,0,1,0
4,0,1,0,0,0,0


We will add it back to the dataset, dropping the dummy for "Unknown".

In [25]:
bd=pd.concat([bd,ab_dummies],1)
bd=bd.drop(["age_band","Unknown"],1)

In [26]:
bd["status"].value_counts()

Partner                 7709
Single/Never Married    1101
Divorced/Separated       679
Widowed                  618
Unknown                   48
Name: status, dtype: int64

In [27]:
bd["st_partner"]=np.where(bd["status"]=="Partner",1,0)
bd["st_singleNm"]=np.where(bd["status"]=="Single/Never Married",1,0)
bd["st_divSep"]=np.where(bd["status"]=="Divorced/Separated",1,0)
bd["st_Wido"]=np.where(bd["status"]=="Widowed",1,0)
bd=bd.drop(["status"],1)

In [28]:
round(bd.groupby("occupation")["y"].mean(),2)

occupation
Business Manager     0.12
Housewife            0.09
Manual Worker        0.11
Other                0.11
Professional         0.12
Retired              0.10
Secretarial/Admin    0.11
Student              0.11
Unknown              0.11
Name: y, dtype: float64

In [29]:
for i in range(len(bd)):
    if bd["occupation"][i] in ["Unknown","Student","Secretarial/Admin","Other","Manual Worker"]:
        bd.loc[i,"occupation"]="oc_11"
    if bd["occupation"][i] in ["Professional","Business Manager"]:
        bd.loc[i,"occupation"]="oc_12"
    if bd["occupation"][i]=="Retired":
        bd.loc[i,"occupation"]="oc_10"
oc_dummies=pd.get_dummies(bd["occupation"])
oc_dummies.head()

,Housewife,oc_10,oc_11,oc_12
0,0,0,1,0
1,0,1,0,0
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1


In [30]:
bd=pd.concat([bd,oc_dummies],1)

bd=bd.drop(["occupation","Housewife"],1)

In [31]:
round(bd.groupby("occupation_partner")["y"].mean(),2)

occupation_partner
Business Manager     0.11
Housewife            0.11
Manual Worker        0.11
Other                0.10
Professional         0.11
Retired              0.10
Secretarial/Admin    0.12
Student              0.12
Unknown              0.10
Name: y, dtype: float64

In [ ]:
# bd["ocp_10"]=0
# bd["ocp_12"]=0
# for i in range(len(bd)):
#     if bd["occupation_partner"][i] in ["Unknown","Retired","Other"]:
#         bd.loc[i,"ocp_10"]=1
#     if bd["occupation_partner"][i] in ["Student","Secretarial/Admin"]:
#         bd.loc[i,"ocp_12"]=1
        
# bd=bd.drop(["occupation_partner","TVarea","post_code","post_area","region"],1)

In [32]:
bd["ocp_11"]=0
bd["ocp_12"]=0
for i in range(len(bd)):
    if bd["occupation_partner"][i] in ["Business Manager","Housewife","Manual Worker", "Professional"]:
        bd.loc[i,"ocp_11"]=1
    if bd["occupation_partner"][i] in ["Student","Secretarial/Admin"]:
        bd.loc[i,"ocp_12"]=1
        
bd=bd.drop(["occupation_partner","TVarea","post_code","post_area","region"],1)

You can see that we have also dropped variables TVarea, region, post_code, post_area. If you look at number of unique values taken by post_area and post_code , you'll realise why decided to drop them. TVarea and region on the other hand we have left for you to make use of and see if using them improves your model.

In [33]:
bd["home_status"].value_counts()

Own Home                9413
Rent from Council/HA     322
Rent Privately           261
Live in Parental Hom     109
Unclassified              50
Name: home_status, dtype: int64

In [34]:
bd["hs_own"]=np.where(bd["home_status"]=="Own Home",1,0)
bd["hs_RentCouncil"]=np.where(bd["home_status"]=="Rent from Council/HA",1,0)
bd["hs_RentPvt"]=np.where(bd["home_status"]=="Rent Privately",1,0)
bd["hs_Parent"]=np.where(bd["home_status"]=="Live in Parental Hom",1,0)
del bd["home_status"]

Notice that we used an alternate syntax for dropping a column here. You can use that too if you like this syntax better.

In [35]:
bd["gender"].value_counts()

Female     7634
Male       2486
Unknown      35
Name: gender, dtype: int64

In [36]:
bd["gender_f"]=np.where(bd["gender"]=="Female",1,0)
del bd["gender"]

In [40]:
bd["self_employed"].value_counts()

No     9436
Yes     719
Name: self_employed, dtype: int64

In [41]:
bd["semp_yes"]=np.where(bd["self_employed"]=="Yes",1,0)
del bd["self_employed"]

In [42]:
bd["self_employed_partner"].value_counts()

No     9026
Yes    1129
Name: self_employed_partner, dtype: int64

In [43]:
bd["semp_part_yes"]=np.where(bd["self_employed_partner"]=="Yes",1,0)
del bd["self_employed_partner"]

In [44]:
bd["family_income"].value_counts()

>=35,000             2517
<27,500, >=25,000    1227
<30,000, >=27,500     994
<25,000, >=22,500     833
<20,000, >=17,500     683
<12,500, >=10,000     677
<17,500, >=15,000     634
<15,000, >=12,500     629
<22,500, >=20,000     590
<10,000, >= 8,000     563
< 8,000, >= 4,000     402
< 4,000               278
Unknown               128
Name: family_income, dtype: int64

We can convert this to number as average of the range once we have figured out what to do with category "Unknown".

In [33]:
round(bd.groupby("family_income")["y"].mean(),4)

family_income
< 4,000              0.0755
< 8,000, >= 4,000    0.0796
<10,000, >= 8,000    0.1066
<12,500, >=10,000    0.1019
<15,000, >=12,500    0.1113
<17,500, >=15,000    0.1230
<20,000, >=17,500    0.1113
<22,500, >=20,000    0.1186
<25,000, >=22,500    0.1032
<27,500, >=25,000    0.0970
<30,000, >=27,500    0.1157
>=35,000             0.1116
Unknown              0.0703
Name: y, dtype: float64

In [47]:
bd["fi"]=4 # by doing this , we have essentially clubbed <4000 and Unknown values . How?
bd.loc[bd["family_income"]=="< 8,000, >= 4,000","fi"]=6
bd.loc[bd["family_income"]=="<10,000, >= 8,000","fi"]=9
bd.loc[bd["family_income"]=="<12,500, >=10,000","fi"]=11.25
bd.loc[bd["family_income"]=="<15,000, >=12,500","fi"]=13.75
bd.loc[bd["family_income"]=="<17,500, >=15,000","fi"]=16.25
bd.loc[bd["family_income"]=="<20,000, >=17,500","fi"]=18.75
bd.loc[bd["family_income"]=="<22,500, >=20,000","fi"]=21.25
bd.loc[bd["family_income"]=="<25,000, >=22,500","fi"]=23.75
bd.loc[bd["family_income"]=="<27,500, >=25,000","fi"]=26.25
bd.loc[bd["family_income"]=="<30,000, >=27,500","fi"]=28.75
bd.loc[bd["family_income"]==">=35,000","fi"]=35
bd=bd.drop(["family_income"],1)

In [53]:
bd.dtypes

REF_NO                               int64
children                             int64
year_last_moved                      int64
Average Credit Card Transaction    float64
Balance Transfer                   float64
Term Deposit                       float64
Life Insurance                     float64
Medical Insurance                  float64
Average A/C Balance                float64
Personal Loan                      float64
Investment in Mutual Fund          float64
Investment Tax Saving Bond         float64
Home Loan                          float64
Online Purchase Amount             float64
Investment in Commudity            float64
Investment in Equity               float64
Investment in Derivative           float64
Portfolio Balance                  float64
y                                    int32
ab_10                                uint8
ab_11                                uint8
ab_13                                uint8
ab_17                                uint8
ab_9       

Now that the entire data is of numeric type, lets beging our modelling process after removing nas from the data.

In [38]:
bd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10155 entries, 0 to 10154
Data columns (total 41 columns):
REF_NO                             10155 non-null int64
children                           10155 non-null int64
year_last_moved                    10155 non-null int64
Average Credit Card Transaction    10155 non-null float64
Balance Transfer                   10155 non-null float64
Term Deposit                       10155 non-null float64
Life Insurance                     10155 non-null float64
Medical Insurance                  10155 non-null float64
Average A/C Balance                10155 non-null float64
Personal Loan                      10155 non-null float64
Investment in Mutual Fund          10155 non-null float64
Investment Tax Saving Bond         10155 non-null float64
Home Loan                          10155 non-null float64
Online Purchase Amount             10155 non-null float64
Investment in Commudity            10155 non-null float64
Investment in Equity       

In [54]:
bd.isnull().sum()

REF_NO                             0
children                           0
year_last_moved                    0
Average Credit Card Transaction    0
Balance Transfer                   0
Term Deposit                       0
Life Insurance                     0
Medical Insurance                  0
Average A/C Balance                0
Personal Loan                      0
Investment in Mutual Fund          0
Investment Tax Saving Bond         0
Home Loan                          0
Online Purchase Amount             0
Investment in Commudity            0
Investment in Equity               0
Investment in Derivative           0
Portfolio Balance                  0
y                                  0
ab_10                              0
ab_11                              0
ab_13                              0
ab_17                              0
ab_9                               0
st_partner                         0
st_singleNm                        0
st_divSep                          0
s

In [55]:
#bd.dropna(axis=0,inplace=True)
bd_train, bd_test = train_test_split(bd, test_size = 0.2,random_state=2)

In [56]:
x_train=bd_train.drop(["y","REF_NO"],1)
y_train=bd_train["y"]
x_test=bd_test.drop(["y","REF_NO"],1)
y_test=bd_test["y"]

In [57]:
pd.crosstab(y_test, columns='N', normalize=True)

col_0,N
y,
0,0.88971
1,0.11029


In [58]:
pd.crosstab(y_train, columns='N', normalize=True)

col_0,N
y,
0,0.893895
1,0.106105


In [59]:
logr=LogisticRegression(penalty="l1",class_weight="balanced",random_state=2)

In [60]:
logr.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=2,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [72]:
list(zip(x_train.columns,logr.coef_[0]))

[('children', 0.050059722600136164),
 ('year_last_moved', 6.2493245282194051e-05),
 ('Average Credit Card Transaction', 0.021522803803976882),
 ('Balance Transfer', -0.00683561926476744),
 ('Term Deposit', -0.020402649374118991),
 ('Life Insurance', 0.011754250278967818),
 ('Medical Insurance', -0.012823144832634852),
 ('Average A/C Balance', -0.004530335725094029),
 ('Personal Loan', -0.034025292274581495),
 ('Investment in Mutual Fund', -0.0026865122367877694),
 ('Investment Tax Saving Bond', 0.088365557915074819),
 ('Home Loan', -0.080138824967718153),
 ('Online Purchase Amount', 0.054932213741778917),
 ('Investment in Commudity', 0.0080863710517619491),
 ('Investment in Equity', 0.0072150045975166678),
 ('Investment in Derivative', 0.0084509367831513641),
 ('Portfolio Balance', -0.00011935283641419788),
 ('ab_10', 0.0),
 ('ab_11', -0.24898153937320278),
 ('ab_13', -0.097421897752742959),
 ('ab_17', 0.49041785665354992),
 ('ab_9', -0.092356635002152521),
 ('st_partner', 0.0),
 ('st_

In [75]:
logr.intercept_

array([-2.59420117])

In [80]:
# score model performance on the test data
roc_auc_score(y_test,logr.predict(x_test))

0.89820835639180963

To arrive at the eventual 1,0 prediction , we need to find some way [some cutoff ] to convert predicted probabilites into two classes. Lets first get the probabilities out.

In [81]:
logr.predict_proba(x_train)

array([[  9.60108018e-01,   3.98919822e-02],
       [  9.60251245e-01,   3.97487553e-02],
       [  9.85573821e-01,   1.44261787e-02],
       ..., 
       [  8.70176233e-01,   1.29823767e-01],
       [  8.57643110e-01,   1.42356890e-01],
       [  6.15304319e-08,   9.99999938e-01]])

In [83]:
prob_score=pd.Series(list(zip(*logr.predict_proba(x_train)))[1])

In [84]:
prob_score

0       0.039892
1       0.039749
2       0.014426
3       0.316042
4       0.039819
5       0.192097
6       0.027130
7       0.021143
8       0.003828
9       1.000000
10      0.188904
11      0.021951
12      0.019005
13      0.031588
14      0.022181
15      0.001281
16      0.000076
17      0.149224
18      0.017280
19      0.061680
20      0.000875
21      0.292859
22      0.683799
23      0.018922
24      0.047804
25      0.040386
26      0.155531
27      0.932833
28      0.055093
29      0.000249
          ...   
8094    0.060695
8095    0.084933
8096    0.994735
8097    0.973805
8098    0.999796
8099    0.038169
8100    0.203926
8101    0.072487
8102    0.999999
8103    0.112841
8104    0.236771
8105    0.094227
8106    0.085044
8107    0.245482
8108    0.999996
8109    0.076544
8110    0.150053
8111    0.072799
8112    0.035287
8113    0.007083
8114    0.271755
8115    0.122255
8116    0.395864
8117    0.002761
8118    0.112451
8119    0.005910
8120    0.002943
8121    0.1298

In [89]:
predicted=pd.Series([0]*len(y_train))
predicted[prob_score>0.5]=1
df=pd.DataFrame(list(zip(y_train,predicted)),columns=["Actual","predicted"])

TP=len(df[(df["Actual"]==1) &(df["predicted"]==1) ])
FP=len(df[(df["Actual"]==0) &(df["predicted"]==1) ])
FN=len(df[(df["Actual"]==1) &(df["predicted"]==0) ])
TN=len(df[(df["Actual"]==0) &(df["predicted"]==0) ])
P=TP+FN
N=TN+FP

Precision=TP/(TP+FP)
Recall=TP/P
Accuracy=(TP+TN)/(P+N)

print('Precision - ', Precision)
print('Recall - ', Recall)
print('Accuracy - ', Accuracy)

Precision -  0.5513819985825655
Recall -  0.9025522041763341
Accuracy -  0.9117429837518464


On these scores , we will consider many cutoffs between 0 to 1

In [85]:
cutoffs=np.linspace(0,1,100)
cutoffs

array([ 0.        ,  0.01010101,  0.02020202,  0.03030303,  0.04040404,
        0.05050505,  0.06060606,  0.07070707,  0.08080808,  0.09090909,
        0.1010101 ,  0.11111111,  0.12121212,  0.13131313,  0.14141414,
        0.15151515,  0.16161616,  0.17171717,  0.18181818,  0.19191919,
        0.2020202 ,  0.21212121,  0.22222222,  0.23232323,  0.24242424,
        0.25252525,  0.26262626,  0.27272727,  0.28282828,  0.29292929,
        0.3030303 ,  0.31313131,  0.32323232,  0.33333333,  0.34343434,
        0.35353535,  0.36363636,  0.37373737,  0.38383838,  0.39393939,
        0.4040404 ,  0.41414141,  0.42424242,  0.43434343,  0.44444444,
        0.45454545,  0.46464646,  0.47474747,  0.48484848,  0.49494949,
        0.50505051,  0.51515152,  0.52525253,  0.53535354,  0.54545455,
        0.55555556,  0.56565657,  0.57575758,  0.58585859,  0.5959596 ,
        0.60606061,  0.61616162,  0.62626263,  0.63636364,  0.64646465,
        0.65656566,  0.66666667,  0.67676768,  0.68686869,  0.69

For each of these cutoff , we are going to look at TP,FP,TN,FN values and caluclate KS. Then we'll chose the best cutoff as the one having highest KS.

In [90]:
KS_cut=[]
for cutoff in cutoffs:
    predicted=pd.Series([0]*len(y_train))
    predicted[prob_score>cutoff]=1
    df=pd.DataFrame(list(zip(y_train,predicted)),columns=["real","predicted"])
    TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
    FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
    TN=len(df[(df["real"]==0) &(df["predicted"]==0) ])
    FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
    P=TP+FN
    N=TN+FP
    KS=(TP/P)-(FP/N)
    KS_cut.append(KS)

cutoff_data=pd.DataFrame(list(zip(cutoffs,KS_cut)),columns=["cutoff","KS"])

KS_cutoff=cutoff_data[cutoff_data["KS"]==cutoff_data["KS"].max()]["cutoff"]

In [91]:
cutoff_data

,cutoff,KS
0,0.000000,0.000000
1,0.010101,0.119351
2,0.020202,0.184092
3,0.030303,0.247515
4,0.040404,0.304682
5,0.050505,0.364132
6,0.060606,0.435463
7,0.070707,0.492413
8,0.080808,0.539391
9,0.090909,0.569314


In [92]:
KS_cutoff

40    0.40404
Name: cutoff, dtype: float64

In [93]:
cutoff_data.loc[35:45]

,cutoff,KS
35,0.353535,0.807422
36,0.363636,0.808957
37,0.373737,0.810355
38,0.383838,0.813384
39,0.393939,0.816552
40,0.404040,0.819189
41,0.414141,0.816889
42,0.424242,0.816576
43,0.434343,0.817619
44,0.444444,0.819017


Now we'll see how this model with the cutoff determined here , performs on the test data.

In [94]:
# Performance on test data
prob_score_test=pd.Series(list(zip(*logr.predict_proba(x_test)))[1])

predicted_test=pd.Series([0]*len(y_test))
predicted_test[prob_score_test>float(KS_cutoff)]=1

df_test=pd.DataFrame(list(zip(y_test,predicted_test)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
print('confusion matrix :\n \n ',k)
TN=k.iloc[0,0]
TP=k.iloc[1,1]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

confusion matrix :
 
  predicted     0    1
real                
0          1624  183
1            25  199


In [95]:
# predicted_test
# score model performance on the test data
roc_auc_score(y_test,predicted_test)

0.89356001462566215

In [96]:
# Accuracy of test
(TP+TN)/(P+N)

0.89758739537173804

In [97]:
# Sensitivity on test
TP/P

0.8883928571428571

In [98]:
# Precision
TP/(TP+FP)

0.52094240837696337

In [76]:
#Specificity on test
TN/N

0.89872717210846709

Next we see how cutoff determined by F_beta score performs on test data for beta values : 0.5,1,2

In [101]:
cutoffs=np.linspace(0.010,0.99,100)
def Fbeta_perf(beta,cutoffs,y_train,prob_score):
    FB_cut=[]
    for cutoff in cutoffs:
        predicted=pd.Series([0]*len(y_train))
        predicted[prob_score>cutoff]=1
        df=pd.DataFrame(list(zip(y_train,predicted)),columns=["real","predicted"])

        TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
        FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
        FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
        P=TP+FN
        
        
        Precision=TP/(TP+FP)
        Recall=TP/P
        FB=(1+beta**2)*Precision*Recall/((beta**2)*Precision+Recall)
        FB_cut.append(FB)

    cutoff_data=pd.DataFrame(list(zip(cutoffs,FB_cut)),columns=["cutoff","FB"])

    FB_cutoff=cutoff_data[cutoff_data["FB"]==cutoff_data["FB"].max()]["cutoff"]
    print('Cut-off - ', float(FB_cutoff))
    
    prob_score_test=pd.Series(list(zip(*logr.predict_proba(x_test)))[1])

    predicted_test=pd.Series([0]*len(y_test))
    predicted_test[prob_score_test>float(FB_cutoff)]=1
    

    df_test=pd.DataFrame(list(zip(y_test,predicted_test)),columns=["real","predicted"])

    k=pd.crosstab(df_test['real'],df_test["predicted"])
    
    #     print('confusion matrix :\n \n ',k)
    TN=k.iloc[0,0]
    TP=k.iloc[1,1]
    FP=k.iloc[0,1]
    FN=k.iloc[1,0]
    P=TP+FN
    N=TN+FP
    print('For beta :',beta)
    print('Accuracy is :',(TP+TN)/(P+N))
    print('Sensitivity is :',(TP/P))
    print('Precision is :',(TP/(TP+FP)))
    print('Specificity is :',(TN/N))
    print('AUC is :',roc_auc_score(y_test,predicted_test))
    print('\n \n \n')
    
    return df_test, 

In [102]:
Fbeta_perf(0.5,cutoffs,y_train,prob_score)
Fbeta_perf(1,cutoffs,y_train,prob_score)
Fbeta_perf(2,cutoffs,y_train,prob_score)

Cut-off -  0.8712121212121212
For beta : 0.5
Accuracy is : 0.939931068439
Sensitivity is : 0.625
Precision is : 0.786516853933
Specificity is : 0.978970669618
AUC is : 0.801985334809

 
 

Cut-off -  0.693030303030303
For beta : 1
Accuracy is : 0.932545544067
Sensitivity is : 0.758928571429
Precision is : 0.671936758893
Specificity is : 0.954067515219
AUC is : 0.856498043324

 
 

Cut-off -  0.5742424242424242
For beta : 2
Accuracy is : 0.929098966027
Sensitivity is : 0.848214285714
Precision is : 0.633333333333
Specificity is : 0.939125622579
AUC is : 0.893669954147

 
 



(      real  predicted
 0        0          0
 1        0          0
 2        0          0
 3        0          0
 4        0          0
 5        0          1
 6        0          0
 7        0          0
 8        1          1
 9        0          0
 10       0          0
 11       0          0
 12       0          0
 13       0          0
 14       0          0
 15       0          0
 16       1          1
 17       0          0
 18       0          0
 19       0          0
 20       0          0
 21       0          0
 22       0          0
 23       0          0
 24       0          0
 25       0          0
 26       0          0
 27       0          0
 28       0          0
 29       1          1
 ...    ...        ...
 2001     0          0
 2002     0          1
 2003     0          1
 2004     0          1
 2005     0          0
 2006     0          0
 2007     0          0
 2008     0          0
 2009     0          0
 2010     0          0
 2011     0          0
 2012     0

In [103]:
final_output = Fbeta_perf(1,cutoffs,y_train,prob_score)

Cut-off -  0.693030303030303
For beta : 1
Accuracy is : 0.932545544067
Sensitivity is : 0.758928571429
Precision is : 0.671936758893
Specificity is : 0.954067515219
AUC is : 0.856498043324

 
 



In [104]:
final_output

(      real  predicted
 0        0          0
 1        0          0
 2        0          0
 3        0          0
 4        0          0
 5        0          1
 6        0          0
 7        0          0
 8        1          1
 9        0          0
 10       0          0
 11       0          0
 12       0          0
 13       0          0
 14       0          0
 15       0          0
 16       1          1
 17       0          0
 18       0          0
 19       0          0
 20       0          0
 21       0          0
 22       0          0
 23       0          0
 24       0          0
 25       0          0
 26       0          0
 27       0          0
 28       0          0
 29       1          1
 ...    ...        ...
 2001     0          0
 2002     0          0
 2003     0          1
 2004     0          1
 2005     0          0
 2006     0          0
 2007     0          0
 2008     0          0
 2009     0          0
 2010     0          0
 2011     0          0
 2012     0

You can see that low beta < 1 favors Specificity where as beta > 1 favors sensitivity

We'll conclude our discussion here. Please do the practice exercises . If you face any issue we'll discuss that either in class or QA forum on LMS.

Prepared By : Lalit Sachan (lalit.sachan@edvancer.in)

In case of any doubts or errata alert; please take to QA forum for discussion.

Doubts will be discussed in live class sessions too. [This doesnt apply for self paced students]